In [ ]:
from init_notebook import *

In [ ]:
img = VF.to_tensor(PIL.Image.open("/home/bergi/Pictures/__diverse/capitalism2.jpg"))
img.shape

In [ ]:
from src.functional import soft_histogram
images = torch.cat([img[None, :, :100, :100], img[None, :, 100:200, :100]])
images = images.view(2 * 3, 100, 100)
h = soft_histogram(images, 128, 0, 1, sigma=100)
h = h.view(2, 3, 128).mean(0)
px.line(h.T)

In [ ]:
from experiments.datasets.classic import _dataset
from src.datasets import WrapDataset
import torchvision
from functools import partial

def flowers102_dataset(
        train: bool,
        shape: Tuple[int, int, int] = (3, 96, 96),
        interpolation: bool = True,
) -> Dataset:
    ds = torchvision.datasets.Flowers102(
        "~/prog/data/datasets/", split="train" if train else "test", #download=True,
    )
    def cropper(item):
        return image_resize_crop(
            item, 
            shape=shape[-2:], 
            interpolation=VF.InterpolationMode.BILINEAR if interpolation else VF.InterpolationMode.NEAREST,
        )
        
    return (
        WrapDataset(ds)
        .transform([
            VF.to_tensor,
            cropper,
        ])
    )

ds = flowers102_dataset(True)
VF.to_pil_image(ds[0][0])

In [ ]:
VF.to_pil_image(make_grid(
    [ds[i][0] for i in range(8*8)]
))

In [ ]:
imgf = torch.fft.fft2(img)
imgf.shape, imgf.real.min(), imgf.real.max(), imgf.imag.min(), imgf.imag.max()

In [ ]:
img2 = ds[0][0]
img2f = torch.fft.fft2(img2)
img2f.shape, img2f.real.min(), img2f.real.max(), img2f.imag.min(), img2f.imag.max()

In [ ]:
display(px.line((img2f[0, :10].real.abs().pow(1/4) * img2f[0, :10].real.sign()).T))
#imgf[0, 10].real.abs().log()
display(px.line((img2f[0, :10].real).T))
#imgf[0, 10].real.abs().log()

In [ ]:
def doit(img):
    y = torch.fft.fft2(img) #.abs() #.log()
    S = 3.
    print(y.shape, y.real.min(), y.real.max(), y.imag.min(), y.imag.max())
    n = img.shape[-2] ** 2
    y.real = (y.real / n).abs().pow(1./S) * y.real.sign()
    #mi = y.real.min().clamp_min(y.imag.min())
    #n = math.prod(img2f.shape[-2:])
    #y = (y + mi) / n 
    print(y.shape, y.real.min(), y.real.max(), y.imag.min(), y.imag.max())
    display(px.imshow(y[0].real))
    display(px.line(y[0, :10].real.T))
    y.real = y.real.abs().pow(S) * y.real.sign() * n
    r = torch.fft.ifft2(y).real #.exp()
    print(r.shape, r.min(), r.max())
    display(VF.to_pil_image(r))

doit(img2)

In [ ]:
def normalized_fft2(
    image: torch.Tensor,
    exponent: float = 3.,
):
    size = image.shape[-1] * image.shape[-2]
    
    y = torch.fft.fft2(image)
    y_r = (y.real / size).abs().pow(1. / exponent) * y.real.sign()
    y_i = (y.imag / size).abs().pow(1. / exponent) * y.imag.sign()

    return torch.cat([y_r, y_i], dim=-3)


def normalized_ifft2(
    y: torch.Tensor,
    exponent: float = 3.,
):
    assert y.shape[-3] % 2 == 0, f"Expected channels divisable by 2, got {y.shape}"
    
    chan = y.shape[-3] // 2
    size = y.shape[-1] * y.shape[-2]

    y_sign = y.sign()
    y = y.abs().pow(exponent) * size
    y = torch.complex(
        y[..., :chan, :, :] * y_sign[..., :chan, :, :],
        y[..., chan:, :, :] * y_sign[..., chan:, :, :],
    )
    image = torch.fft.ifft2(y).real

    return image


f = normalized_fft2(img)
i = normalized_ifft2(f)      
VF.to_pil_image(i)